In [2]:
#!jupyter nbconvert --to=python plot_Chlorophyll_FESOM_python3.ipynb

[NbConvertApp] Converting notebook plot_Chlorophyll_FESOM_python3.ipynb to python
[NbConvertApp] Writing 3288 bytes to plot_Chlorophyll_FESOM_python3.py


In [ ]:
class plot_Chl:
    '''
    class plot_Chl(resultpath,varname,years,mesh,runid,depth,mapproj,label,units,convert=False)
    
    convert: set True to convert FESOM output [mmol ChlA/m3] to [mg ChlA/m3]
    '''
    
    def __init__(self,resultpath,varname,years,mesh,runid,depth,mapproj,label,units,convert=False):
        
        self.resultpath = resultpath
        self.varname = varname
        self.years = years
        self.mesh = mesh
        self.runid = runid
        self.depth = depth
        self.mapproj = mapproj
        self.label = label
        self.units = units
        self.convert = convert
        
        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import cartopy.crs as ccrs

        import pyfesom2 as pf
                
        if(self.mapproj == 'rob'): self.mapproj = ccrs.Robinson()
        elif(self.mapproj == 'pc'): self.mapproj = ccrs.PlateCarree()
        else: raise ValueError('Map projection not supported!')
            
        
        Chl = pf.get_data(self.resultpath, self.varname, 
                     #[last_year-10,last_year],
                     self.years,
                     self.mesh, runid=self.runid, how="mean", compute=True, depth=self.depth)

        # convert FESOM CHl.a data -------------------------------------------------------------------------------------
        #Chl.a : 893.509 g/mol
        if self.convert:
            conv = 893.509
            Chl = Chl * conv
            self.units = 'Chl.a [mg m$^{-3}$]'
            print('\n***\nFESOM data was CONVERTED, assuming [mmol ChlA/m3] to [mg ChlA/m3]\n***')
        else:
            print('\n***\nFESOM data was NOT CONVERTED, assuming [mg ChlA/m3]\n***')
    
        # interpolation
        lat         = np.arange(-89.5,90.,1.)
        lon         = np.arange(-179.5,180.,1.)
        latdic, londic = np.meshgrid(lat, lon)

        Chl_interp = pf.fesom2regular(
                        data = Chl,
                        mesh = mesh,
                        lons = londic, 
                        lats = latdic)

        # plotting
        levels = np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                                   0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                                   1,2,3,4,5,7])
        ticks = [0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3,5,7]
        ticks_label = ['0','0.01','0.03','0.05','0.07','0.1','0.3','0.5','0.7','1','3','5','7']    


        if(mapproj == 'rob'): 
            projection = ccrs.Robinson()
        elif(mapproj == 'pc'): 
            projection = ccrs.PlateCarree()
        else: 
            print('mapproj not supported')

        fig, axes = plt.subplots(1,1, 
                 subplot_kw=dict(projection=projection),
                 gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                 figsize=(15,12))

        m2 = axes
        f2 = m2.pcolormesh(londic, latdic, Chl_interp, 
                           transform = ccrs.PlateCarree(),
                           norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
        
        m2.set_title(label, fontsize=16)

        m2.coastlines(resolution='110m', color='black', linewidth=1)

        # add one colorbar for first row plots below figure
        cbar = fig.colorbar(f2,
                            ax = axes, 
                            #location ='bottom',
                            orientation = 'horizontal',
                            ticks = ticks,
                            fraction=0.1, pad=0.1) 
        #cbar.ax.tick_params(labelsize=14)
        cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
        cbar.set_label(units, fontsize=16)